In [ ]:
!lscpu

Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          48 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   8
  On-line CPU(s) list:    0-7
Vendor ID:                AuthenticAMD
  Model name:             AMD EPYC 7B12
    CPU family:           23
    Model:                49
    Thread(s) per core:   2
    Core(s) per socket:   4
    Socket(s):            1
    Stepping:             0
    BogoMIPS:             4499.99
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge m
                          ca cmov pat pse36 clflush mmx fxsr sse sse2 ht syscall
                           nx mmxext fxsr_opt pdpe1gb rdtscp lm constant_tsc rep
                          _good nopl nonstop_tsc cpuid extd_apicid tsc_known_fre
                          q pni pclmulqdq ssse3 fma cx16 sse4_1 sse4_2 x2apic mo
                          vbe popcnt aes xsave avx f16c rdrand hypervisor lahf_l

**-----------Load The Data------------**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import time
from collections import deque
from tqdm import tqdm
from typing import Dict, List, Tuple
import math
import random
import numpy as np
from numpy import ndarray

In [ ]:
# Set seeds for reproducibility
# --- Fixed parameters ---
dim = 7 # dimension of the points
np.random.seed(42)  # set seed for reproducibility

In [ ]:
# Define the dataset path
dataset_path = '/content/drive/MyDrive/datasets/dataSearch/higgs_ds_mod_orig.txt'

# Load the dataset
# If the file has a different delimiter (e.g., commas), you can set delimiter=',' accordingly.
dataset = np.loadtxt(dataset_path)
total_points = len(dataset)

In [ ]:
# Data sizes:
SB = 10_000    # Small Batch: 10K points
MB = 100_000   # Medium Batch: 100K points
LB = 1_000_000 # Large Batch: 1M points

# Query sizes:
SQ = 20        # Small Query: 20 queries
MQ = 100       # Medium Query: 100 queries
LQ = 500       # Large Query: 500 queries

# === Define the test combinations ===
# Each tuple is of the form (data_size, query_size)
test_combinations = [
    (LB, MQ)
]

In [23]:
# ----------------------------
# Phase 1: Pre-generate and Store Queries from the Dataset
# ----------------------------

queries_list = []
st_idx = 0  # Starting index for slicing the dataset

for data_size, query_size in test_combinations:
    # Slice the current data chunk
    end_idx = st_idx + data_size
    current_data = dataset[st_idx:end_idx]
    st_idx = end_idx  # Update start index for next chunk

    # Ensure query selection is different for each batch size
    if query_size <= len(current_data):  # Sampling from the current batch only
        random_indices = np.random.choice(len(current_data), size=query_size, replace=False)
        queries = current_data[random_indices]  # Select query points from the current batch
    else:
        raise ValueError(f"Query size {query_size} exceeds available data points {len(current_data)}.")

    # Append the selected queries to the list
    queries_list.append(queries)


**--------QuickSelect Algorithm----------------**

In [ ]:
from collections import deque
from typing import List, Tuple

#*-----------------------------------------------------------------------------
def quickselect(distances:List, k:int):
    """QuickSelect algorithm to find the k-th smallest distance."""

    if len(distances) == 1:
        return distances[0]

    if len(distances) == 0:
        return None

    stack = deque()
    stack.append( (distances, k) )
    while stack:

        distances, k = stack.pop()
        if len(distances) == 1:
            return distances[0]

        # Returned as a tuple
        pivot = median_of_medians(distances)
        lows, highs, pivots = partition(distances, pivot)

        if len(lows) > k:
            stack.append((lows, k))
        elif len(lows) + len(pivots) > k:
            return pivot
        else:
            stack.append((highs, k - len(lows) - len(pivots)))
    return pivot

#*-----------------------------------------------------------------------------
def partition(distances, pivot):
    lows, highs, pivots = [], [], []

    for d in distances:
        if d[0] < pivot[0]:
            lows.append(d)
        elif d[0] > pivot[0]:
            highs.append(d)
        else:
            pivots.append(d)
    return lows, highs, pivots

#*-----------------------------------------------------------------------------
def median_of_medians(distances, grp_size=5):
    """Find the index of the median distance in a list of distances.
    Returns the index and the median distance as a tuple."""

    stack = deque()
    # Split the list in chunks of grp_size
    # And add the lists to the stack on the right side
    for i in range(0, len(distances), grp_size):
        subList = distances[i:i+grp_size]
        insertion_sort(subList)
        stack.append(subList)

    # We want to find the median of the medians
    # and return the index of that median
    median_list = []
    stack_count = len(stack)
    while 0 < stack_count:

        subList = stack.pop()
        stack_count -= 1

        mid_index = len(subList) // 2
        median_list.append(subList[mid_index])

        if len(median_list) == grp_size:
            insertion_sort(median_list)
            # append to the left, since we are popping from the right
            stack.appendleft(median_list)
            if stack_count == 0:
                stack_count = len(stack)
            median_list = []
        # If the count is 0, we have reached the end of the old stack
        # and need to insert the remaining elements,
        # stack count and the median list will be reset
        if stack_count == 0 and len(median_list) > 0:
            if len(median_list) > 1:
                insertion_sort(median_list)
            stack.appendleft(median_list)
            stack_count = len(stack)
            if stack_count == 1 and len(median_list) == 1:
                return median_list[0]
            median_list = []

#*-----------------------------------------------------------------------------
def insertion_sort(arr):
# In-place insertion sort, efficient for small arrays
    for i in range(1, len(arr)):
        key = arr[i]
        j = i - 1
        while j >= 0 and arr[j][0] > key[0]:
            arr[j + 1] = arr[j]
            j -= 1
        arr[j + 1] = key
#*-----------------------------------------------------------------------------

**----------KD Tree---------------------**

In [ ]:
import numpy as np
from collections import deque
from typing import List, Tuple, Union

#*-----------------------------------------------------------------------------
class KDNode:
    def __init__(self, points=None, left=None, right=None, parent=None):
        self.points = points if points is not None else []  # Now always a list of points
        self.split_dim = None
        self.split_val = None
        self.leaf = False
        self.left_tr = left
        self.right_tr = right
        self.parent = parent
        self.centroid = None
        self.radius = 0
        # NumPy array cache for search operations
        self._np_points = None

#*-----------------------------------------------------------------------------
class KDTree:
    def __init__(self, leaf_size=40, epsilon=0.1):
        self.points = None
        self.root = None
        self.dim = None
        self.leaf_size = leaf_size
        self.epsilon = epsilon

    #*-----------------------------------------------------------------------------
    def find_best_split_dimension(self, points: np.ndarray, start_dim: int) -> Tuple[int, bool]:
        if len(points) == 0:
            return 0, True
        for d in range(self.dim):
            curr_dim = (start_dim + d) % self.dim
            if not np.all(points[:, curr_dim] == points[0, curr_dim]):
                return curr_dim, False
        return 0, True

    #*-----------------------------------------------------------------------------
    def build(self, points=None):
        self.points = points
        self.dim = points.shape[1]
        self.root = KDNode()
        stack = deque([(self.root, points, 0)])

        while stack:
            node, node_points, depth = stack.pop()

            if len(node_points) <= self.leaf_size:
                node.leaf = True
                node.points = node_points.tolist() # Convert to list of points
                node.centroid = np.mean(node_points, axis=0)
                if len(node_points) > 1:
                    node.radius = np.max(np.linalg.norm(node_points - node.centroid, axis=1))
                continue

            split_dim, all_equal = self.find_best_split_dimension(node_points, depth % self.dim)
            if all_equal:
                node.leaf = True
                node.points = node_points.tolist()
                node.centroid = np.mean(node_points, axis=0)
                if len(node_points) > 1:
                    node.radius = np.max(np.linalg.norm(node_points - node.centroid, axis=1))
                continue

            node.split_dim = split_dim
            coords = [(p[split_dim], i) for i, p in enumerate(node_points)]
            k = len(node_points) // 2
            med_ = quickselect(coords, k)
            median_val = med_[0]
            node.split_val = median_val

            left_points = node_points[node_points[:, split_dim] < (median_val - self.epsilon)]
            right_points = node_points[node_points[:, split_dim] > (median_val + self.epsilon)]
            middle_points = node_points[
                (node_points[:, split_dim] >= (median_val - self.epsilon)) &
                (node_points[:, split_dim] <= (median_val + self.epsilon))
            ]

            node.points = [p for p in middle_points]
            if len(middle_points) > 0:
                node.centroid = np.mean(middle_points, axis=0)
                if len(middle_points) > 1:
                    node.radius = np.max(np.linalg.norm(middle_points - node.centroid, axis=1))

            if len(left_points) > 0:
                node.left_tr = KDNode(parent=node)
                stack.append((node.left_tr, left_points, depth + 1))
            if len(right_points) > 0:
                node.right_tr = KDNode(parent=node)
                stack.append((node.right_tr, right_points, depth + 1))
            if len(left_points) == 0 and len(right_points) == 0:
                node.leaf = True

        return self.root

    #*-----------------------------------------------------------------------------
    def insert(self, point: Union[List, np.ndarray]):
        if self.root is None:
            self.dim = len(point)
            self.points = [point]
            self.root = KDNode()
            self.root.points = [point]
            self.root.centroid = point
            self.root.radius = 0
            self.root.leaf = True
            return

        current = self.root

        while not current.leaf:
            split_dim = current.split_dim
            split_val = current.split_val

            if (len(current.points) > 0 and len(current.points) < self.leaf_size
                and abs(point[split_dim] - split_val) <= self.epsilon):
                n = len(current.points)
                current.points.append(point)
                current.centroid = (n * current.centroid + point) / (n + 1)
                dist_to_centroid = np.linalg.norm(point - current.centroid)
                current.radius = max(current.radius, dist_to_centroid)
                # Clear the NumPy cache since we modified the points
                current._np_points = None
                return

            if point[split_dim] < split_val - self.epsilon:
                if current.left_tr is None:
                    current.left_tr = KDNode(parent=current)
                    current.left_tr.points = [point]
                    current.left_tr.centroid = point
                    current.left_tr.radius = 0
                    current.left_tr.leaf = True
                    return
                current = current.left_tr
            else:
                if current.right_tr is None:
                    current.right_tr = KDNode(parent=current)
                    current.right_tr.points = [point]
                    current.right_tr.centroid = point
                    current.right_tr.radius = 0
                    current.right_tr.leaf = True
                    return
                current = current.right_tr

        n = len(current.points)

        current.points.append(point)
        current.centroid = (n * current.centroid + point) / (n + 1)
        dist_to_centroid = np.linalg.norm(point - current.centroid)
        current.radius = max(current.radius, dist_to_centroid)
        # Clear the NumPy cache since we modified the points
        current._np_points = None

    #*-----------------------------------------------------------------------------
    def prepare_for_search(self):
        """
        Convert all list-based point collections to numpy arrays for efficient search.
        This should be called once after all insertions and before searches.
        """

        stack = [self.root]

        while stack:
            node = stack.pop()

            if node is None:
                continue

            # Convert points to numpy array
            if node.points:
                node._np_points = np.array(node.points)

            # Add child nodes to stack
            if node.left_tr:
                stack.append(node.left_tr)
            if node.right_tr:
                stack.append(node.right_tr)

    #*-----------------------------------------------------------------------------
    def search(self, query):
        """
        Search for the nearest neighbor of the query point using KD-tree.
        Uses triangle inequality pruning at leaf nodes.

        Parameters:
            query (np.ndarray): The query point

        Returns:
            Tuple of (distance to nearest neighbor, nearest neighbor point)
        """

        best_dist = float('inf')
        best_point = None

        # Stack for depth-first traversal
        stack = [self.root]

        while stack:
            node = stack.pop()

            if node.leaf:
                dist_to_centroid = np.linalg.norm(query - node.centroid)
                if dist_to_centroid - node.radius >= best_dist:
                    continue

                # Check if _np_points exists, if not, create it
                if node._np_points is None and node.points:
                    node._np_points = np.array(node.points)

                # Use the cached NumPy array for distance calculations
                if node._np_points is not None:
                    dists = np.linalg.norm(node._np_points - query, axis=1)
                    min_idx = np.argmin(dists)
                    if dists[min_idx] < best_dist:
                        best_dist = dists[min_idx]
                        best_point = node._np_points[min_idx]
                continue

            if node.split_dim is not None:
                if len(node.points) > 0:
                    # Check if _np_points exists, if not, create it
                    if node._np_points is None and node.points:
                        node._np_points = np.array(node.points)

                    # Use the cached NumPy array for distance calculations
                    if node._np_points is not None:
                        dists = np.linalg.norm(node._np_points - query, axis=1)
                        min_idx = np.argmin(dists)
                        if dists[min_idx] < best_dist:
                            best_dist = dists[min_idx]
                            best_point = node._np_points[min_idx]

                split_dim = node.split_dim
                split_val = node.split_val
                diff = query[split_dim] - split_val

                if diff < -self.epsilon:
                    if node.right_tr and abs(diff) < best_dist:
                        stack.append(node.right_tr)
                    if node.left_tr:
                        stack.append(node.left_tr)
                elif diff > self.epsilon:
                    if node.left_tr and abs(diff) < best_dist:
                        stack.append(node.left_tr)
                    if node.right_tr:
                        stack.append(node.right_tr)
                else:
                    if abs(diff) <= self.epsilon:
                        if diff < 0:
                            if node.left_tr:
                                stack.append(node.left_tr)
                            if node.right_tr:
                                stack.append(node.right_tr)
                        else:
                            if node.right_tr:
                                stack.append(node.right_tr)
                            if node.left_tr:
                                stack.append(node.left_tr)

        return best_dist, best_point
#*-----------------------------------------------------------------------------

**-------KD Tree Node Size Optimization For Rebuild-------**

**--------------Run 1---------------------**

In [ ]:
import time
from tqdm import tqdm

# Test code for KD Tree
results = []

# Define leaf size range
leaf_size_range_start = 50
leaf_size_range_end = 2_500  # Ensure inclusive range
leaf_size_step = 50

# Fix the epsilon at 0.0
epsilon=0.0

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data = dataset[:data_size]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, leaf_size in enumerate(range(leaf_size_range_start, leaf_size_range_end + leaf_size_step, leaf_size_step)):
    print(f"\n{idx+1}: === Testing with Leaf Size: {leaf_size:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # --- Build the KD Tree ---
    build_start = time.perf_counter()
    kdtree = KDTree(leaf_size=leaf_size, epsilon=epsilon)
    kdtree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Leaf Size {leaf_size:,})"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    search_time = search_end - search_start

    total_time = build_time + search_time
    results.append((leaf_size, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Leaf Size: {leaf_size:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of KD Tree Node Size Test ===")
for l_size, b_time, s_time, t_time in results:
    print(f"Leaf Size: {l_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Leaf Size: 50, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Leaf Size 50): 100%|██████████| 100/100 [00:00<00:00, 381.08it/s]


-------------------------------------------------------------
Build Time: 25.9232 sec, Search Time: 0.8342 sec
Leaf Size: 50, Total Time: 26.7575 sec

2: === Testing with Leaf Size: 100, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Leaf Size 100): 100%|██████████| 100/100 [00:00<00:00, 379.51it/s]


-------------------------------------------------------------
Build Time: 24.8735 sec, Search Time: 0.8918 sec
Leaf Size: 100, Total Time: 25.7653 sec

3: === Testing with Leaf Size: 150, Data Size: 1,000,000 points, Query Size: 100 queries ===


...

-------------------------------------------------------------
Build Time: 17.3847 sec, Search Time: 0.7840 sec
Leaf Size: 2,400, Total Time: 18.1686 sec

49: === Testing with Leaf Size: 2,450, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Leaf Size 2,450): 100%|██████████| 100/100 [00:00<00:00, 383.44it/s]


-------------------------------------------------------------
Build Time: 17.3765 sec, Search Time: 0.7861 sec
Leaf Size: 2,450, Total Time: 18.1626 sec

50: === Testing with Leaf Size: 2,500, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Leaf Size 2,500): 100%|██████████| 100/100 [00:00<00:00, 379.82it/s]

-------------------------------------------------------------
Build Time: 17.3770 sec, Search Time: 0.7889 sec
Leaf Size: 2,500, Total Time: 18.1658 sec

=== Summary of KD Tree Node Size Test ===
Leaf Size: 50, Build Time: 25.9232 sec, Search Time: 0.8342 sec, Total Time: 26.7575 sec
Leaf Size: 100, Build Time: 24.8735 sec, Search Time: 0.8918 sec, Total Time: 25.7653 sec
Leaf Size: 150, Build Time: 23.6970 sec, Search Time: 0.7909 sec, Total Time: 24.4879 sec
Leaf Size: 200, Build Time: 23.1169 sec, Search Time: 0.7869 sec, Total Time: 23.9038 sec
Leaf Size: 250, Build Time: 22.3778 sec, Search Time: 0.7712 sec, Total Time: 23.1491 sec
Leaf Size: 300, Build Time: 21.3817 sec, Search Time: 0.7759 sec, Total Time: 22.1576 sec
Leaf Size: 350, Build Time: 21.6686 sec, Search Time: 0.7729 sec, Total Time: 22.4415 sec
Leaf Size: 400, Build Time: 21.2105 sec, Search Time: 0.7647 sec, Total Time: 21.9752 sec
Leaf Size: 450, Build Time: 20.5002 sec, Search Time: 0.7553 sec, Total Time: 21.2555

**-------------Run 2-----------------------**

In [ ]:
import time
from tqdm import tqdm

# Test code for KD Tree
results = []

# Define leaf size range
leaf_size_range_start = 50
leaf_size_range_end = 2_500  # Ensure inclusive range
leaf_size_step = 50

# Fix the epsilon at 0.0
epsilon=0.0

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data = dataset[:data_size]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, leaf_size in enumerate(range(leaf_size_range_start, leaf_size_range_end + leaf_size_step, leaf_size_step)):
    print(f"\n{idx+1}: === Testing with Leaf Size: {leaf_size:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # --- Build the KD Tree ---
    build_start = time.perf_counter()
    kdtree = KDTree(leaf_size=leaf_size, epsilon=epsilon)
    kdtree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Leaf Size {leaf_size:,})"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    search_time = search_end - search_start

    total_time = build_time + search_time
    results.append((leaf_size, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Leaf Size: {leaf_size:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of KD Tree Node Size Test ===")
for l_size, b_time, s_time, t_time in results:
    print(f"Leaf Size: {l_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Leaf Size: 50, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Leaf Size 50): 100%|██████████| 100/100 [00:00<00:00, 382.44it/s]


-------------------------------------------------------------
Build Time: 25.4473 sec, Search Time: 0.7802 sec
Leaf Size: 50, Total Time: 26.2275 sec

2: === Testing with Leaf Size: 100, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Leaf Size 100): 100%|██████████| 100/100 [00:00<00:00, 393.61it/s]


-------------------------------------------------------------
Build Time: 24.0552 sec, Search Time: 0.7513 sec
Leaf Size: 100, Total Time: 24.8065 sec

3: === Testing with Leaf Size: 150, Data Size: 1,000,000 points, Query Size: 100 queries ===


...

-------------------------------------------------------------
Build Time: 17.6069 sec, Search Time: 0.8065 sec
Leaf Size: 2,400, Total Time: 18.4134 sec

49: === Testing with Leaf Size: 2,450, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Leaf Size 2,450): 100%|██████████| 100/100 [00:00<00:00, 362.60it/s]


-------------------------------------------------------------
Build Time: 17.6372 sec, Search Time: 0.8060 sec
Leaf Size: 2,450, Total Time: 18.4432 sec

50: === Testing with Leaf Size: 2,500, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Leaf Size 2,500): 100%|██████████| 100/100 [00:00<00:00, 374.90it/s]

-------------------------------------------------------------
Build Time: 17.6834 sec, Search Time: 0.7955 sec
Leaf Size: 2,500, Total Time: 18.4789 sec

=== Summary of KD Tree Node Size Test ===
Leaf Size: 50, Build Time: 25.4473 sec, Search Time: 0.7802 sec, Total Time: 26.2275 sec
Leaf Size: 100, Build Time: 24.0552 sec, Search Time: 0.7513 sec, Total Time: 24.8065 sec
Leaf Size: 150, Build Time: 22.9909 sec, Search Time: 0.7750 sec, Total Time: 23.7659 sec
Leaf Size: 200, Build Time: 22.7540 sec, Search Time: 0.7769 sec, Total Time: 23.5309 sec
Leaf Size: 250, Build Time: 21.3555 sec, Search Time: 0.7718 sec, Total Time: 22.1273 sec
Leaf Size: 300, Build Time: 21.0950 sec, Search Time: 0.7593 sec, Total Time: 21.8543 sec
Leaf Size: 350, Build Time: 21.0899 sec, Search Time: 0.7608 sec, Total Time: 21.8506 sec
Leaf Size: 400, Build Time: 21.0694 sec, Search Time: 0.7638 sec, Total Time: 21.8332 sec
Leaf Size: 450, Build Time: 20.5184 sec, Search Time: 0.7657 sec, Total Time: 21.2841

**-------------Opt Node Size For Rebuild-------------**

In [ ]:
import numpy as np

# Leaf sizes
leaf_sizes = np.array([  50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700,
  750, 800, 850, 900, 950,1000,1050,1100,1150,1200,1250,1300,1350,1400,
 1450,1500,1550,1600,1650,1700,1750,1800,1850,1900,1950,2000,2050,2100,
 2150,2200,2250,2300,2350,2400,2450,2500])

# Total times from two runs (or versions)
total_times_1 = np.array([26.7575,25.7653,24.4879,23.9038,23.1491,22.1576,22.4415,21.9752,21.2555,
 20.852 ,20.9176,20.7013,20.5122,20.7754,20.4821,20.2833,20.3746,19.74  ,
 19.9571,19.5679,19.3432,19.2332,19.2038,19.3884,19.4536,19.2178,19.2697,
 19.4182,19.2881,19.2366,19.4255,19.0682,19.1373,19.0919,19.0175,18.4163,
 18.5512,18.5679,18.1808,18.1126,18.3262,18.0852,18.3918,17.9702,18.3124,
 18.0886,18.2287,18.1686,18.1626,18.1658])

total_times_2 = np.array([26.2275,24.8065,23.7659,23.5309,22.1273,21.8543,21.8506,21.8332,21.2841,
 20.9116,20.8585,20.9607,20.9818,20.9537,21.397 ,21.2606,20.809 ,20.4568,
 20.3204,19.6369,19.7695,20.0034,19.6656,20.3406,20.1724,19.9756,20.1786,
 20.4814,19.8888,19.8448,19.8138,19.6291,19.9102,19.7759,19.6203,19.1252,
 18.9038,18.6514,18.506 ,18.5073,18.4981,18.4302,18.3048,18.4495,18.4328,
 18.3673,18.8038,18.4134,18.4432,18.4789])

# Average total time across the two runs
avg_total_times = (total_times_1 + total_times_2) / 2

# Index of the minimum average total time
min_idx = np.argmin(avg_total_times)

# Corresponding leaf size
optimal_leaf_size = leaf_sizes[min_idx]

# Print results
print(f"Average total time (sec): {avg_total_times[min_idx]:.4f}")
print(f"Minimum found at leaf size: {optimal_leaf_size}")


Average total time (sec): 18.2098
Minimum found at leaf size: 2200


**-----Epsilon Optimization For Rebuild------**

**-----------Run 1-------------------**

In [14]:
import time
from tqdm import tqdm

# Test code for KD Tree
results = []

leaf_size = 2200
# Define epsilon range
epsilon_range_start = 0.0
epsilon_range_end = 0.1
epsilon_size_step = 0.001

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data = dataset[:data_size]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, epsilon in enumerate(np.arange(epsilon_range_start, epsilon_range_end + epsilon_size_step, epsilon_size_step)):
    epsilon = round(epsilon, 10)
    print(f"\n{idx+1}: === Testing with Epsilon: {epsilon:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # --- Build the KD Tree ---
    build_start = time.perf_counter()
    kdtree = KDTree(leaf_size=leaf_size, epsilon=epsilon)
    kdtree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Leaf Size {leaf_size:,})"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    search_time = search_end - search_start

    total_time = build_time + search_time
    results.append((epsilon, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Epsilon: {epsilon:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of KD Tree Epsilon Size Test ===")
for e_size, b_time, s_time, t_time in results:
    print(f"Epsilon: {e_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Epsilon: 0.0, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Leaf Size 2,200): 100%|██████████| 100/100 [00:00<00:00, 532.39it/s]


-------------------------------------------------------------
Build Time: 13.3855 sec, Search Time: 0.5181 sec
Epsilon: 0.0, Total Time: 13.9036 sec

2: === Testing with Epsilon: 0.001, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Leaf Size 2,200): 100%|██████████| 100/100 [00:00<00:00, 563.18it/s]


-------------------------------------------------------------
Build Time: 12.7410 sec, Search Time: 0.4998 sec
Epsilon: 0.001, Total Time: 13.2408 sec

3: === Testing with Epsilon: 0.002, Data Size: 1,000,000 points, Query Size: 100 queries ===


...

-------------------------------------------------------------
Build Time: 8.7904 sec, Search Time: 1.2882 sec
Epsilon: 0.098, Total Time: 10.0786 sec

100: === Testing with Epsilon: 0.099, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Leaf Size 2,200): 100%|██████████| 100/100 [00:01<00:00, 89.85it/s]


-------------------------------------------------------------
Build Time: 8.9802 sec, Search Time: 1.3469 sec
Epsilon: 0.099, Total Time: 10.3270 sec

101: === Testing with Epsilon: 0.1, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Leaf Size 2,200): 100%|██████████| 100/100 [00:01<00:00, 91.65it/s]

-------------------------------------------------------------
Build Time: 9.2607 sec, Search Time: 1.3185 sec
Epsilon: 0.1, Total Time: 10.5792 sec

=== Summary of KD Tree Epsilon Size Test ===
Epsilon: 0.0, Build Time: 13.3855 sec, Search Time: 0.5181 sec, Total Time: 13.9036 sec
Epsilon: 0.001, Build Time: 12.7410 sec, Search Time: 0.4998 sec, Total Time: 13.2408 sec
Epsilon: 0.002, Build Time: 12.7041 sec, Search Time: 0.5118 sec, Total Time: 13.2159 sec
Epsilon: 0.003, Build Time: 12.7764 sec, Search Time: 0.5109 sec, Total Time: 13.2873 sec
Epsilon: 0.004, Build Time: 12.9191 sec, Search Time: 0.5263 sec, Total Time: 13.4454 sec
Epsilon: 0.005, Build Time: 12.6092 sec, Search Time: 0.5511 sec, Total Time: 13.1603 sec
Epsilon: 0.006, Build Time: 12.4744 sec, Search Time: 0.5465 sec, Total Time: 13.0209 sec
Epsilon: 0.007, Build Time: 12.1986 sec, Search Time: 0.5585 sec, Total Time: 12.7571 sec
Epsilon: 0.008, Build Time: 12.2854 sec, Search Time: 0.5728 sec, Total Time: 12.8583 se

**----------Run 2--------------------**

In [16]:
import time
from tqdm import tqdm

# Test code for KD Tree
results = []

leaf_size = 2200
# Define epsilon range
epsilon_range_start = 0.0
epsilon_range_end = 0.1
epsilon_size_step = 0.001

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data = dataset[:data_size]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, epsilon in enumerate(np.arange(epsilon_range_start, epsilon_range_end + epsilon_size_step, epsilon_size_step)):
    epsilon = round(epsilon, 10)
    print(f"\n{idx+1}: === Testing with Epsilon: {epsilon:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # --- Build the KD Tree ---
    build_start = time.perf_counter()
    kdtree = KDTree(leaf_size=leaf_size, epsilon=epsilon)
    kdtree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Leaf Size {leaf_size:,})"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    search_time = search_end - search_start

    total_time = build_time + search_time
    results.append((epsilon, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Epsilon: {epsilon:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of KD Tree Epsilon Size Test ===")
for e_size, b_time, s_time, t_time in results:
    print(f"Epsilon: {e_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Epsilon: 0.0, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Leaf Size 2,200): 100%|██████████| 100/100 [00:00<00:00, 546.36it/s]


-------------------------------------------------------------
Build Time: 13.1123 sec, Search Time: 0.5037 sec
Epsilon: 0.0, Total Time: 13.6160 sec

2: === Testing with Epsilon: 0.001, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Leaf Size 2,200): 100%|██████████| 100/100 [00:00<00:00, 577.60it/s]


-------------------------------------------------------------
Build Time: 13.1479 sec, Search Time: 0.5133 sec
Epsilon: 0.001, Total Time: 13.6612 sec

3: === Testing with Epsilon: 0.002, Data Size: 1,000,000 points, Query Size: 100 queries ===


...

-------------------------------------------------------------
Build Time: 9.9030 sec, Search Time: 1.3847 sec
Epsilon: 0.098, Total Time: 11.2877 sec

100: === Testing with Epsilon: 0.099, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Leaf Size 2,200): 100%|██████████| 100/100 [00:01<00:00, 86.68it/s]


-------------------------------------------------------------
Build Time: 9.9044 sec, Search Time: 1.4067 sec
Epsilon: 0.099, Total Time: 11.3110 sec

101: === Testing with Epsilon: 0.1, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Leaf Size 2,200): 100%|██████████| 100/100 [00:01<00:00, 82.95it/s]

-------------------------------------------------------------
Build Time: 10.0185 sec, Search Time: 1.4486 sec
Epsilon: 0.1, Total Time: 11.4670 sec

=== Summary of KD Tree Epsilon Size Test ===
Epsilon: 0.0, Build Time: 13.1123 sec, Search Time: 0.5037 sec, Total Time: 13.6160 sec
Epsilon: 0.001, Build Time: 13.1479 sec, Search Time: 0.5133 sec, Total Time: 13.6612 sec
Epsilon: 0.002, Build Time: 13.3937 sec, Search Time: 0.5087 sec, Total Time: 13.9024 sec
Epsilon: 0.003, Build Time: 13.1051 sec, Search Time: 0.5223 sec, Total Time: 13.6274 sec
Epsilon: 0.004, Build Time: 14.1339 sec, Search Time: 0.5297 sec, Total Time: 14.6635 sec
Epsilon: 0.005, Build Time: 13.4852 sec, Search Time: 0.5690 sec, Total Time: 14.0542 sec
Epsilon: 0.006, Build Time: 14.2437 sec, Search Time: 0.5588 sec, Total Time: 14.8024 sec
Epsilon: 0.007, Build Time: 13.1368 sec, Search Time: 0.5795 sec, Total Time: 13.7163 sec
Epsilon: 0.008, Build Time: 13.7906 sec, Search Time: 0.6048 sec, Total Time: 14.3954 s

**--------Epsilon Opt Value For Rebuild------------**

In [17]:
import numpy as np

# Epsilon sizes
epsilon_sizes = np.array([0.   ,0.001,0.002,0.003,0.004,0.005,0.006,0.007,0.008,0.009,0.01 ,0.011,
 0.012,0.013,0.014,0.015,0.016,0.017,0.018,0.019,0.02 ,0.021,0.022,0.023,
 0.024,0.025,0.026,0.027,0.028,0.029,0.03 ,0.031,0.032,0.033,0.034,0.035,
 0.036,0.037,0.038,0.039,0.04 ,0.041,0.042,0.043,0.044,0.045,0.046,0.047,
 0.048,0.049,0.05 ,0.051,0.052,0.053,0.054,0.055,0.056,0.057,0.058,0.059,
 0.06 ,0.061,0.062,0.063,0.064,0.065,0.066,0.067,0.068,0.069,0.07 ,0.071,
 0.072,0.073,0.074,0.075,0.076,0.077,0.078,0.079,0.08 ,0.081,0.082,0.083,
 0.084,0.085,0.086,0.087,0.088,0.089,0.09 ,0.091,0.092,0.093,0.094,0.095,
 0.096,0.097,0.098,0.099,0.1  ])

# Total times from two runs (or versions)
total_times_1 = np.array([13.9036,13.2408,13.2159,13.2873,13.4454,13.1603,13.0209,12.7571,12.8583,
 12.9089,12.9015,12.9741,12.8724,13.2448,12.7423,12.7265,12.6842,12.6001,
 12.5931,12.5026,12.442 ,12.5446,12.5418,12.4431,12.304 ,12.2394,12.2627,
 12.5243,12.4107,12.015 ,12.1928,12.2258,12.1281,12.0499,11.8434,11.9902,
 11.8162,11.8076,12.0189,11.7311,11.9495,11.7995,11.8525,12.0187,11.6292,
 11.8667,11.78  ,11.9407,12.0617,11.8921,12.1031,11.606 ,11.698 ,11.8855,
 11.6005,11.6045,11.4865,12.1126,12.8426,12.1613,11.8777,12.1671,11.0683,
 11.6771,11.0754,10.9004,11.1139,10.8838,10.7564,10.9118,10.9822,10.9358,
 10.601 ,10.6356,10.7532,10.5449,10.5624,10.4837,10.3294,10.3691,10.555 ,
 10.3557,10.5005,10.4493,10.672 ,10.5526,10.5437,10.7706,10.8156,10.5951,
 10.4505,10.6233,10.3094,10.3982,10.2749,10.3178,10.2305,10.4357,10.0786,
 10.327 ,10.5792])

total_times_2 = np.array( [13.616 ,13.6612,13.9024,13.6274,14.6635,14.0542,14.8024,13.7163,14.3954,
 13.6428,14.1792,14.1875,13.7878,13.5174,13.8676,13.8017,13.8145,13.5691,
 13.7635,13.8575,13.5534,13.5933,13.2351,13.1403,13.8916,13.151 ,13.577 ,
 13.1045,13.2862,13.24  ,13.3159,13.046 ,13.3155,12.9151,12.4009,12.86  ,
 12.4832,12.655 ,12.3644,12.4172,12.528 ,12.4519,12.2395,12.7038,12.1038,
 12.3952,12.1192,12.2275,12.2502,13.5268,13.3636,12.6543,12.21  ,11.9099,
 12.2795,11.8768,12.0055,11.9215,11.693 ,11.6892,11.6006,11.4751,11.6243,
 11.3744,11.5186,11.2572,11.4652,11.5186,11.201 ,11.307 ,11.1211,10.9682,
 11.1979,11.1122,10.9707,11.271 ,11.0261,11.1096,11.0302,10.8428,10.7657,
 10.8864,10.8987,10.7545,10.937 ,10.7284,10.8573,10.92  ,11.0397,10.7672,
 10.876 ,10.7695,10.7229,10.7823,11.1576,11.4505,11.2626,11.675 ,11.2877,
 11.311 ,11.467 ])

# Average total time across the two runs
avg_total_times = (total_times_1 + total_times_2) / 2

# Index of the minimum average total time
min_idx = np.argmin(avg_total_times)

# Corresponding leaf size
optimal_epsilon_size = epsilon_sizes[min_idx]

# Print results
print(f"Average total time (sec): {avg_total_times[min_idx]:.4f}")
print(f"Minimum found at epsilon size: {optimal_epsilon_size}")


Average total time (sec): 10.5161
Minimum found at epsilon size: 0.092


**------Node Optimization For Insert-----------**

**---------------Run 1-----------------------**

In [18]:
import time
from tqdm import tqdm

# Test code for KD Tree
results = []

# Define leaf size range
leaf_size_range_start = 50
leaf_size_range_end = 2500  # Ensure inclusive range
leaf_size_step = 50

leaf_size_rebuld = 2200
epsilon_rebuild = 0.092

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data_rebuild = dataset[:MB]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, leaf_size in enumerate(range(leaf_size_range_start, leaf_size_range_end + leaf_size_step, leaf_size_step)):
    print(f"\n{idx+1}: === Testing with Leaf Size: {leaf_size:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # Build the tree with 100K points, using the optimized n_s, and epsilon
    kdtree = KDTree(leaf_size=leaf_size_rebuld, epsilon=epsilon_rebuild)
    kdtree.build(current_data_rebuild)

    # Obtain Dataset Sample
    current_data = dataset[MB:data_size+MB]
    # --- Build the KD Tree With Insertions---
    build_start = time.perf_counter()
    kdtree.leaf_size = leaf_size
    for pt in tqdm(current_data, desc="Inserting KD Tree"):
        kdtree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Leaf Size {leaf_size:,})"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    search_time = search_end - search_start

    total_time = build_time + search_time
    results.append((leaf_size, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Leaf Size: {leaf_size:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of KD Tree Node Size Test ===")
for l_size, b_time, s_time, t_time in results:
    print(f"Leaf Size: {l_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Leaf Size: 50, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Leaf Size 50): 100%|██████████| 100/100 [00:00<00:00, 126.74it/s]


-------------------------------------------------------------
Build Time: 7.5347 sec, Search Time: 1.5006 sec
Leaf Size: 50, Total Time: 9.0353 sec

2: === Testing with Leaf Size: 100, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Leaf Size 100): 100%|██████████| 100/100 [00:00<00:00, 123.93it/s]


-------------------------------------------------------------
Build Time: 7.7378 sec, Search Time: 1.5476 sec
Leaf Size: 100, Total Time: 9.2854 sec

3: === Testing with Leaf Size: 150, Data Size: 1,000,000 points, Query Size: 100 queries ===


...

-------------------------------------------------------------
Build Time: 7.8496 sec, Search Time: 1.5284 sec
Leaf Size: 2,400, Total Time: 9.3780 sec

49: === Testing with Leaf Size: 2,450, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Leaf Size 2,450): 100%|██████████| 100/100 [00:00<00:00, 124.21it/s]


-------------------------------------------------------------
Build Time: 7.9790 sec, Search Time: 1.5341 sec
Leaf Size: 2,450, Total Time: 9.5131 sec

50: === Testing with Leaf Size: 2,500, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Leaf Size 2,500): 100%|██████████| 100/100 [00:00<00:00, 124.57it/s]

-------------------------------------------------------------
Build Time: 7.8823 sec, Search Time: 1.5196 sec
Leaf Size: 2,500, Total Time: 9.4019 sec

=== Summary of KD Tree Node Size Test ===
Leaf Size: 50, Build Time: 7.5347 sec, Search Time: 1.5006 sec, Total Time: 9.0353 sec
Leaf Size: 100, Build Time: 7.7378 sec, Search Time: 1.5476 sec, Total Time: 9.2854 sec
Leaf Size: 150, Build Time: 7.7597 sec, Search Time: 1.5767 sec, Total Time: 9.3364 sec
Leaf Size: 200, Build Time: 7.7026 sec, Search Time: 1.5224 sec, Total Time: 9.2250 sec
Leaf Size: 250, Build Time: 7.6652 sec, Search Time: 1.5712 sec, Total Time: 9.2364 sec
Leaf Size: 300, Build Time: 7.6649 sec, Search Time: 1.5326 sec, Total Time: 9.1975 sec
Leaf Size: 350, Build Time: 7.7271 sec, Search Time: 1.6430 sec, Total Time: 9.3702 sec
Leaf Size: 400, Build Time: 7.8551 sec, Search Time: 1.5229 sec, Total Time: 9.3780 sec
Leaf Size: 450, Build Time: 7.7615 sec, Search Time: 1.4881 sec, Total Time: 9.2496 sec
Leaf Size: 500,

**-------------Run 2-----------------------**

In [20]:
import time
from tqdm import tqdm

# Test code for KD Tree
results = []

# Define leaf size range
leaf_size_range_start = 50
leaf_size_range_end = 2500  # Ensure inclusive range
leaf_size_step = 50

leaf_size_rebuld = 2200
epsilon_rebuild = 0.092

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data_rebuild = dataset[:MB]  # Directly slicing for clarity



# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, leaf_size in enumerate(range(leaf_size_range_start, leaf_size_range_end + leaf_size_step, leaf_size_step)):
    print(f"\n{idx+1}: === Testing with Leaf Size: {leaf_size:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # Build the tree with 100K points, using the optimized n_s, and epsilon
    kdtree = KDTree(leaf_size=leaf_size_rebuld, epsilon=epsilon_rebuild)
    kdtree.build(current_data_rebuild)

    # Readjust for the insertion
    data_size, query_size = test_combinations[0]
    # Obtain Dataset Sample
    current_data = dataset[MB:data_size+MB]
    # --- Build the KD Tree With Insertions---
    build_start = time.perf_counter()
    kdtree.leaf_size = leaf_size
    for pt in tqdm(current_data, desc="Inserting KD Tree"):
        kdtree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Leaf Size {leaf_size:,})"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    search_time = search_end - search_start

    total_time = build_time + search_time
    results.append((leaf_size, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Leaf Size: {leaf_size:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of KD Tree Node Size Test ===")
for l_size, b_time, s_time, t_time in results:
    print(f"Leaf Size: {l_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Leaf Size: 50, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Leaf Size 50): 100%|██████████| 100/100 [00:00<00:00, 132.28it/s]


-------------------------------------------------------------
Build Time: 7.6100 sec, Search Time: 1.4917 sec
Leaf Size: 50, Total Time: 9.1016 sec

2: === Testing with Leaf Size: 100, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Leaf Size 100): 100%|██████████| 100/100 [00:00<00:00, 127.04it/s]


-------------------------------------------------------------
Build Time: 7.6760 sec, Search Time: 1.5389 sec
Leaf Size: 100, Total Time: 9.2149 sec

3: === Testing with Leaf Size: 150, Data Size: 1,000,000 points, Query Size: 100 queries ===


...

-------------------------------------------------------------
Build Time: 8.1183 sec, Search Time: 1.6139 sec
Leaf Size: 2,400, Total Time: 9.7322 sec

49: === Testing with Leaf Size: 2,450, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Leaf Size 2,450): 100%|██████████| 100/100 [00:00<00:00, 125.43it/s]


-------------------------------------------------------------
Build Time: 8.0227 sec, Search Time: 1.5611 sec
Leaf Size: 2,450, Total Time: 9.5839 sec

50: === Testing with Leaf Size: 2,500, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Leaf Size 2,500): 100%|██████████| 100/100 [00:00<00:00, 123.45it/s]

-------------------------------------------------------------
Build Time: 8.1984 sec, Search Time: 1.6028 sec
Leaf Size: 2,500, Total Time: 9.8011 sec

=== Summary of KD Tree Node Size Test ===
Leaf Size: 50, Build Time: 7.6100 sec, Search Time: 1.4917 sec, Total Time: 9.1016 sec
Leaf Size: 100, Build Time: 7.6760 sec, Search Time: 1.5389 sec, Total Time: 9.2149 sec
Leaf Size: 150, Build Time: 7.9438 sec, Search Time: 1.6003 sec, Total Time: 9.5441 sec
Leaf Size: 200, Build Time: 7.5840 sec, Search Time: 1.5218 sec, Total Time: 9.1058 sec
Leaf Size: 250, Build Time: 7.6792 sec, Search Time: 1.5536 sec, Total Time: 9.2328 sec
Leaf Size: 300, Build Time: 7.6191 sec, Search Time: 1.5221 sec, Total Time: 9.1412 sec
Leaf Size: 350, Build Time: 7.8909 sec, Search Time: 1.5380 sec, Total Time: 9.4290 sec
Leaf Size: 400, Build Time: 7.5316 sec, Search Time: 1.5646 sec, Total Time: 9.0962 sec
Leaf Size: 450, Build Time: 7.8189 sec, Search Time: 1.5318 sec, Total Time: 9.3507 sec
Leaf Size: 500,

**----Node Size Optimization For Insertion---------**

In [21]:
import numpy as np

# Leaf sizes
leaf_sizes = np.array([  50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700,
  750, 800, 850, 900, 950,1000,1050,1100,1150,1200,1250,1300,1350,1400,
 1450,1500,1550,1600,1650,1700,1750,1800,1850,1900,1950,2000,2050,2100,
 2150,2200,2250,2300,2350,2400,2450,2500])

# Total times from two runs (or versions)
total_times_1 = np.array( [9.1016,9.2149,9.5441,9.1058,9.2328,9.1412,9.429 ,9.0962,9.3507,9.257 ,
 9.1313,9.2704,9.2086,9.3649,9.377 ,9.3333,9.3051,9.2669,9.158 ,9.4435,
 9.6402,9.3708,9.3902,9.3037,9.4262,9.1941,9.1232,9.4763,9.4454,9.4049,
 9.4276,9.3225,9.4931,9.3416,9.4125,9.3345,9.679 ,9.333 ,9.3418,9.3499,
 9.5092,9.5826,9.4479,9.6314,9.349 ,9.3449,9.3655,9.7322,9.5839,9.8011])

total_times_2 = np.array([9.0353,9.2854,9.3364,9.225 ,9.2364,9.1975,9.3702,9.378 ,9.2496,9.2335,
 9.2211,9.2274,9.2768,9.1705,9.1484,8.9987,9.2974,9.2982,9.2814,9.1796,
 9.0459,9.2333,9.3019,8.9733,9.1377,9.1827,9.4077,9.3994,9.3195,9.3653,
 9.2364,9.4347,9.3763,9.4892,9.3356,9.3936,9.2805,9.4265,9.4664,9.3471,
 9.3401,9.3635,9.4059,9.3621,9.2291,9.186 ,9.2933,9.378 ,9.5131,9.4019])

# Average total time across the two runs
avg_total_times = (total_times_1 + total_times_2) / 2

# Index of the minimum average total time
min_idx = np.argmin(avg_total_times)

# Corresponding leaf size
optimal_leaf_size = leaf_sizes[min_idx]

# Print results
print(f"Average total time (sec): {avg_total_times[min_idx]:.4f}")
print(f"Minimum found at leaf size: {optimal_leaf_size}")


Average total time (sec): 9.0684
Minimum found at leaf size: 50


**------------Epsilon Optimization for Insertion-------------**

**-------------------Run 1---------------------------**

In [22]:
import time
from tqdm import tqdm

# Test code for KD Tree
results = []

leaf_size_rebuild = 2200
epsilon_rebuild = 0.092

leaf_size_insert = 50

# Define epsilon range
epsilon_range_start = 0.0
epsilon_range_end = 0.1
epsilon_size_step = 0.001

# Obtain Dataset Sample
current_data_rebuild = dataset[:MB]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, epsilon in enumerate(np.arange(epsilon_range_start, epsilon_range_end + epsilon_size_step, epsilon_size_step)):
    epsilon = round(epsilon, 10)
    print(f"\n{idx+1}: === Testing with Epsilon: {epsilon:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # Build the tree with 100K points, using the optimized n_s, and epsilon
    kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
    kdtree.build(current_data_rebuild)

    # Readjust for the insertion
    data_size, query_size = test_combinations[0]
    # Obtain Dataset Sample
    current_data = dataset[MB:data_size+MB]

    kdtree.leaf_size = leaf_size_insert
    kdtree.epsilon = epsilon

    # --- Build the KD Tree With Insertions---
    build_start = time.perf_counter()
    for pt in tqdm(current_data, desc="Inserting KD Tree"):
        kdtree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Leaf Size {leaf_size:,})"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    search_time = search_end - search_start

    total_time = build_time + search_time
    results.append((epsilon, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Epsilon: {epsilon:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of KD Tree Epsilon Size Test ===")
for e_size, b_time, s_time, t_time in results:
    print(f"Epsilon: {e_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Epsilon: 0.0, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Leaf Size 2,500): 100%|██████████| 100/100 [00:00<00:00, 155.67it/s]


-------------------------------------------------------------
Build Time: 7.7161 sec, Search Time: 1.3807 sec
Epsilon: 0.0, Total Time: 9.0968 sec

2: === Testing with Epsilon: 0.001, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Leaf Size 2,500): 100%|██████████| 100/100 [00:00<00:00, 149.41it/s]


-------------------------------------------------------------
Build Time: 7.6481 sec, Search Time: 1.4176 sec
Epsilon: 0.001, Total Time: 9.0657 sec

3: === Testing with Epsilon: 0.002, Data Size: 1,000,000 points, Query Size: 100 queries ===


...

-------------------------------------------------------------
Build Time: 7.5828 sec, Search Time: 1.4911 sec
Epsilon: 0.098, Total Time: 9.0739 sec

100: === Testing with Epsilon: 0.099, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Leaf Size 2,500): 100%|██████████| 100/100 [00:00<00:00, 125.13it/s]


-------------------------------------------------------------
Build Time: 7.7149 sec, Search Time: 1.5616 sec
Epsilon: 0.099, Total Time: 9.2765 sec

101: === Testing with Epsilon: 0.1, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Leaf Size 2,500): 100%|██████████| 100/100 [00:00<00:00, 131.44it/s]

-------------------------------------------------------------
Build Time: 7.6108 sec, Search Time: 1.5091 sec
Epsilon: 0.1, Total Time: 9.1199 sec

=== Summary of KD Tree Epsilon Size Test ===
Epsilon: 0.0, Build Time: 7.7161 sec, Search Time: 1.3807 sec, Total Time: 9.0968 sec
Epsilon: 0.001, Build Time: 7.6481 sec, Search Time: 1.4176 sec, Total Time: 9.0657 sec
Epsilon: 0.002, Build Time: 7.6718 sec, Search Time: 1.4145 sec, Total Time: 9.0863 sec
Epsilon: 0.003, Build Time: 7.7822 sec, Search Time: 1.3924 sec, Total Time: 9.1746 sec
Epsilon: 0.004, Build Time: 7.5310 sec, Search Time: 1.3755 sec, Total Time: 8.9065 sec
Epsilon: 0.005, Build Time: 7.3988 sec, Search Time: 1.4261 sec, Total Time: 8.8249 sec
Epsilon: 0.006, Build Time: 7.2884 sec, Search Time: 1.4038 sec, Total Time: 8.6923 sec
Epsilon: 0.007, Build Time: 7.5235 sec, Search Time: 1.4270 sec, Total Time: 8.9505 sec
Epsilon: 0.008, Build Time: 7.5442 sec, Search Time: 1.3915 sec, Total Time: 8.9357 sec
Epsilon: 0.009, B

**----------------Run 2-------------------------**

In [24]:
import time
from tqdm import tqdm

# Test code for KD Tree
results = []

leaf_size_rebuild = 2200
epsilon_rebuild = 0.092

leaf_size_insert = 50

# Define epsilon range
epsilon_range_start = 0.0
epsilon_range_end = 0.1
epsilon_size_step = 0.001

# Obtain Dataset Sample
current_data_rebuild = dataset[:MB]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, epsilon in enumerate(np.arange(epsilon_range_start, epsilon_range_end + epsilon_size_step, epsilon_size_step)):
    epsilon = round(epsilon, 10)
    print(f"\n{idx+1}: === Testing with Epsilon: {epsilon:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # Build the tree with 100K points, using the optimized n_s, and epsilon
    kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
    kdtree.build(current_data_rebuild)

    # Readjust for the insertion
    data_size, query_size = test_combinations[0]
    # Obtain Dataset Sample
    current_data = dataset[MB:data_size+MB]

    kdtree.leaf_size = leaf_size_insert
    kdtree.epsilon = epsilon

    # --- Build the KD Tree With Insertions---
    build_start = time.perf_counter()
    for pt in tqdm(current_data, desc="Inserting KD Tree"):
        kdtree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Leaf Size {leaf_size:,})"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    search_time = search_end - search_start

    total_time = build_time + search_time
    results.append((epsilon, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Epsilon: {epsilon:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of KD Tree Epsilon Size Test ===")
for e_size, b_time, s_time, t_time in results:
    print(f"Epsilon: {e_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Epsilon: 0.0, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Leaf Size 2,500): 100%|██████████| 100/100 [00:00<00:00, 156.32it/s]


-------------------------------------------------------------
Build Time: 7.5562 sec, Search Time: 1.3774 sec
Epsilon: 0.0, Total Time: 8.9336 sec

2: === Testing with Epsilon: 0.001, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Leaf Size 2,500): 100%|██████████| 100/100 [00:00<00:00, 158.98it/s]


-------------------------------------------------------------
Build Time: 7.6753 sec, Search Time: 1.3817 sec
Epsilon: 0.001, Total Time: 9.0571 sec

3: === Testing with Epsilon: 0.002, Data Size: 1,000,000 points, Query Size: 100 queries ===


...

-------------------------------------------------------------
Build Time: 7.6832 sec, Search Time: 1.6532 sec
Epsilon: 0.098, Total Time: 9.3363 sec

100: === Testing with Epsilon: 0.099, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Leaf Size 2,500): 100%|██████████| 100/100 [00:00<00:00, 114.96it/s]


-------------------------------------------------------------
Build Time: 7.8072 sec, Search Time: 1.6600 sec
Epsilon: 0.099, Total Time: 9.4673 sec

101: === Testing with Epsilon: 0.1, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Leaf Size 2,500): 100%|██████████| 100/100 [00:00<00:00, 121.46it/s]

-------------------------------------------------------------
Build Time: 7.5805 sec, Search Time: 1.5716 sec
Epsilon: 0.1, Total Time: 9.1521 sec

=== Summary of KD Tree Epsilon Size Test ===
Epsilon: 0.0, Build Time: 7.5562 sec, Search Time: 1.3774 sec, Total Time: 8.9336 sec
Epsilon: 0.001, Build Time: 7.6753 sec, Search Time: 1.3817 sec, Total Time: 9.0571 sec
Epsilon: 0.002, Build Time: 7.4716 sec, Search Time: 1.3942 sec, Total Time: 8.8658 sec
Epsilon: 0.003, Build Time: 7.4836 sec, Search Time: 1.3826 sec, Total Time: 8.8662 sec
Epsilon: 0.004, Build Time: 7.5182 sec, Search Time: 1.3807 sec, Total Time: 8.8989 sec
Epsilon: 0.005, Build Time: 7.5600 sec, Search Time: 1.3837 sec, Total Time: 8.9436 sec
Epsilon: 0.006, Build Time: 7.7081 sec, Search Time: 1.4307 sec, Total Time: 9.1388 sec
Epsilon: 0.007, Build Time: 7.8460 sec, Search Time: 1.3867 sec, Total Time: 9.2327 sec
Epsilon: 0.008, Build Time: 7.5999 sec, Search Time: 1.4009 sec, Total Time: 9.0009 sec
Epsilon: 0.009, B

**------Epsilon Optimization For Insertion----**

In [26]:
import numpy as np

# Epsilon sizes
epsilon_sizes = np.array([0.   ,0.001,0.002,0.003,0.004,0.005,0.006,0.007,0.008,0.009,0.01 ,0.011,
 0.012,0.013,0.014,0.015,0.016,0.017,0.018,0.019,0.02 ,0.021,0.022,0.023,
 0.024,0.025,0.026,0.027,0.028,0.029,0.03 ,0.031,0.032,0.033,0.034,0.035,
 0.036,0.037,0.038,0.039,0.04 ,0.041,0.042,0.043,0.044,0.045,0.046,0.047,
 0.048,0.049,0.05 ,0.051,0.052,0.053,0.054,0.055,0.056,0.057,0.058,0.059,
 0.06 ,0.061,0.062,0.063,0.064,0.065,0.066,0.067,0.068,0.069,0.07 ,0.071,
 0.072,0.073,0.074,0.075,0.076,0.077,0.078,0.079,0.08 ,0.081,0.082,0.083,
 0.084,0.085,0.086,0.087,0.088,0.089,0.09 ,0.091,0.092,0.093,0.094,0.095,
 0.096,0.097,0.098,0.099,0.1  ])

# Total times from two runs (or versions)
total_times_1 = np.array([8.9336,9.0571,8.8658,8.8662,8.8989,8.9436,9.1388,9.2327,9.0009,9.1537,
 8.926 ,8.8689,9.0558,8.8904,9.0403,9.1714,8.972 ,8.8825,9.159 ,9.2139,
 9.1788,9.038 ,9.1262,9.0809,9.0607,9.1614,9.0604,9.3412,9.0075,9.2118,
 8.8246,9.0207,9.1392,8.9186,8.9368,9.0915,9.0991,9.073 ,9.2478,9.1469,
 9.2177,9.0036,8.8975,9.2318,9.0126,9.0923,9.3721,9.1229,8.9536,8.9937,
 8.8667,9.1621,9.0573,9.0067,9.1796,8.9303,9.2444,9.1716,9.16  ,9.0339,
 9.2681,9.1311,8.9215,9.1054,9.1202,9.1101,9.3431,9.1635,9.3002,9.2177,
 9.3378,9.0617,9.1467,9.2908,9.0039,9.2635,9.1003,9.2478,9.195 ,9.1502,
 9.1612,9.1261,9.213 ,9.2167,9.3445,9.6321,9.2494,9.2555,9.3592,9.079 ,
 9.0888,9.1635,9.1728,9.0247,9.2639,9.4268,9.2536,9.2307,9.3363,9.4673,
 9.1521])

total_times_2 = np.array( [9.0968,9.0657,9.0863,9.1746,8.9065,8.8249,8.6923,8.9505,8.9357,9.0322,
 8.8841,9.0314,8.8707,8.8429,8.902 ,9.0306,8.9383,9.1932,9.0516,8.9757,
 9.0733,8.9349,8.9717,9.0942,8.9577,9.0813,9.1461,8.9711,8.9488,9.0442,
 8.9992,9.2134,9.1081,9.0521,8.8824,9.0343,8.9907,9.0392,9.4374,9.1493,
 9.001 ,9.1981,9.0477,9.1444,9.2487,9.1099,9.0039,8.9899,8.8724,9.1111,
 9.1533,9.0524,9.2113,9.0586,9.118 ,8.9567,9.1785,9.0719,8.9886,9.0744,
 9.0702,9.2712,8.9585,8.9726,9.2227,8.9272,9.1419,9.1225,9.0928,9.2227,
 9.0639,9.0338,8.9865,9.0888,8.8949,8.9935,8.9448,9.2469,9.0683,9.1781,
 9.367 ,9.0907,9.3855,8.8097,9.0485,8.9797,9.1388,9.0452,9.124 ,8.9764,
 9.1235,9.1181,9.2086,9.0976,9.1683,9.0119,9.0853,9.0217,9.0739,9.2765,
 9.1199])

# Average total time across the two runs
avg_total_times = (total_times_1 + total_times_2) / 2

# Index of the minimum average total time
min_idx = np.argmin(avg_total_times)

# Corresponding leaf size
optimal_epsilon_size = epsilon_sizes[min_idx]

# Print results
print(f"Average total time (sec): {avg_total_times[min_idx]:.4f}")
print(f"Minimum found at epsilon size: {optimal_epsilon_size}")


Average total time (sec): 8.8666
Minimum found at epsilon size: 0.013
